# Advent of Code 2018

## Norvig's Utilities

In [1]:
# Python 3.x Utility Functions

%matplotlib inline
import matplotlib.pyplot as plt

import os
import urllib.request

import re
import numpy as np
import math
import random
import time

from collections import Counter, defaultdict, namedtuple, deque, abc, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, count as count_from)
from heapq       import heappop, heappush
from numba       import jit

letters  = 'abcdefghijklmnopqrstuvwxyz'

cache = lru_cache(None)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

################ Functions for Input, Parsing

def Input(day, year=2018):
    "Open this day's input file."
    directory = 'data/advent{}/'.format(year)
    filename = directory+'input{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        if not os.path.exists(directory):
            os.makedirs(directory)
        #urllib.request.urlretrieve("https://raw.githubusercontent.com/jeffreyrosenbluth/pytudes/master/data/" + filename, filename)
        return Input(day)

def Inputstr(day, year=2018): 
    "The contents of this day's input file as a str."
    return Input(day, year).read().rstrip('\n')
    
def Array(lines):
    "Parse an iterable of str lines into a 2-D array. If `lines` is a str, splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return mapt(Vector, lines)

def Vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(Atom, line.replace(',', ' ').split())

def Integers(text): 
    "Return a tuple of all integers in a string."
    return mapt(int, re.findall(r'-?\b\d+\b', text))

def Atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token
        
def error(err=RuntimeError, *args): raise err(*args)

################ Functions on Iterables

def first(iterable, default=None): 
    "The first item in an iterable, or default if it is empty."
    return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

identity = lambda x: x

def groupby(iterable, key=identity):
    "Return a dict of {key(item): [items...]} grouping all items in iterable by keys."
    groups = defaultdict(list)
    for item in iterable:
        groups[key(item)].append(item)
    return groups

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks:
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def sequence(iterable, type=tuple):
    "Coerce iterable to sequence: leave alone if already a sequence, else make it `type`."
    return iterable if isinstance(iterable, abc.Sequence) else type(iterable)

def join(iterable, sep=''):
    "Join the items in iterable, converting each to a string first."
    return sep.join(map(str, iterable))
                
def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c
            
def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def length(iterable):
    "Same as len(list(iterable)), but without consuming memory."
    return sum(1 for _ in iterable)

def shuffled(iterable):
    "Create a new list out of iterable, and shuffle it."
    new = list(iterable)
    random.shuffle(new)
    return new
    
flatten = chain.from_iterable

################ Functional programming

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def map2d(fn, grid):
    "Apply fn to every element in a 2-dimensional grid."
    return tuple(mapt(fn, row) for row in grid)

def repeat(n, fn, arg, *args, **kwds):
    "Repeat arg = fn(arg) n times, return arg."
    return nth(repeatedly(fn, arg, *args, **kwds), n)

def repeatedly(fn, arg, *args, **kwds):
    "Yield arg, fn(arg), fn(fn(arg)), ..."
    yield arg
    while True:
        arg = fn(arg, *args, **kwds)
        yield arg
        
def compose(f, g): 
    "The function that computes f(g(x))."
    return lambda x: f(g(x))

################ Making immutable objects
            
class Set(frozenset):
    "A frozenset, but with a prettier printer."
    def __repr__(self): return '{' + join(sorted(self), ', ') + '}'
    
def canon(items, typ=None):
    "Canonicalize these order-independent items into a hashable canonical form."
    typ = typ or (cat if isinstance(items, str) else tuple)
    return typ(sorted(items))
            
################ Math Functions
            
def transpose(matrix): return tuple(zip(*matrix))

def isqrt(n):
    "Integer square root (rounds down)."
    return int(n ** 0.5)

def ints(start, end, step=1):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1, step)

def floats(start, end, step=1.0):
    "Yield floats from start to end (inclusive), by increments of step."
    m = (1.0 if step >= 0 else -1.0)
    while start * m <= end * m:
        yield start
        start += step
        
def multiply(numbers):
    "Multiply all the numbers together."
    result = 1
    for n in numbers:
        result *= n
    return result

import operator as op

operations = {'>': op.gt, '>=': op.ge, '==': op.eq,
              '<': op.lt, '<=': op.le, '!=': op.ne,
              '+': op.add, '-': op.sub, '*': op.mul, 
              '/': op.truediv, '**': op.pow}

################ 2-D points implemented using (x, y) tuples

def X(point): return point[0]
def Y(point): return point[1]

origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def turn_right(heading): return HEADINGS[HEADINGS.index(heading) - 1]
def turn_around(heading):return HEADINGS[HEADINGS.index(heading) - 2]
def turn_left(heading):  return HEADINGS[HEADINGS.index(heading) - 3]

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)
            
class Struct:
    "A structure that can have any fields defined."
    def __init__(self, **entries): self.__dict__.update(entries)
    def __repr__(self): 
        fields = ['{}={}'.format(f, self.__dict__[f]) 
                  for f in sorted(self.__dict__)]
        return 'Struct({})'.format(', '.join(fields))

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (where h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

In [35]:
from datetime import datetime, date, time

## Day 1

In [3]:
data1 = mapt(int, Input(1))
sum(data1)

595

### Part 2

In [4]:
def first_repeat(data):
    current, freqs = 0, {0}
    for f in data:
        current += f
        if current in freqs:
            return current
        freqs.add(current)

first_repeat(cycle(data1))

80598

## Day 2

In [5]:
data2 = Vector(Inputstr(2))

def two_three(str):
    c = Set(Counter(str).values())
    return (2 in c, 3 in c)

def check_sum(data): return multiply(reduce(lambda p,t: add(p, two_three(t)), data, (0,0)))

check_sum(data2)

6474

### Part 2

In [6]:
def diff(s1, s2): return sum(x[0] != x[1] for x in zip(s1, s2))

def common(data):
    d = sorted(data)
    a, b = first(p for p in pairwise(d) if diff(p[0], p[1]) == 1)
    return ''.join(a[i] for i in range(len(a)) if a[i] == b[i])
    
common(data2)

'mxhwoglxgeauywfkztndcvjqr'

## Day 3

In [7]:
def parse(data):
    ls = []
    for t in data:
        l = list(t)
        l[3] = int(l[3].rstrip(':'))
        ds = l[4]
        x, y = ds.split('x')
        l[4] = int(x)
        l.append(int(y))
        l.pop(1)
        ls.append(l)
    return ls
        
data3 = parse(Array(Inputstr(3)))

def dim(l): return ((l[1], l[2]), (l[1] + l[3], l[2] + l[4]))

def counts(data):
    ds = map(dim, data)
    c = Counter()
    for d in ds:
        for i in range(d[0][0] + 1, d[1][0] + 1):
            for j in range(d[0][1] + 1, d[1][1] + 1):
                c[(i,j)] += 1
    return sum(1 for x in c.values() if x > 1)
    
counts(data3)

104126

### Part 2

In [8]:
def separate(d1, d2):
    px, py, pw, ph = d1[1], d1[2], d1[3], d1[4]
    qx, qy, qw, qh = d2[1], d2[2], d2[3], d2[4]
    return px > qx + qw or px + pw < qx or py > qy + qh or py + ph < qy
    
def overlaps(d, ds):
    count = 0
    for x in ds:
        if not separate(d, x):
            if count == 1: return True
            else: count += 1
    return False
    
def intact(data):
    for d in data:
        if not overlaps(d, data):
            return d
        

intact(data3)

['#695', 337, 825, 27, 17]

## Day 4

In [34]:
data4 = Inputstr(4).splitlines()
def parse4(t):
    result = []
    t1 = map(lambda x: x.strip('['), t)
    t2 = map(lambda x: x.replace(']', ''), t1)
    t3 = map(lambda x: x.split(), t2)
    for r in t3:
        l = r[0].split('-')
        y, m, d = tuple(map(int,l))
        r[0] = date(y,m,d)
        result.append(r)
    return result
        
    
list(parse4(data4))

[[datetime.date(1518, 6, 2), '23:58', 'Guard', '#179', 'begins', 'shift'],
 [datetime.date(1518, 9, 18), '00:43', 'wakes', 'up'],
 [datetime.date(1518, 6, 6), '00:10', 'falls', 'asleep'],
 [datetime.date(1518, 5, 12), '00:52', 'wakes', 'up'],
 [datetime.date(1518, 7, 2), '00:39', 'wakes', 'up'],
 [datetime.date(1518, 7, 21), '23:50', 'Guard', '#2917', 'begins', 'shift'],
 [datetime.date(1518, 9, 22), '00:47', 'wakes', 'up'],
 [datetime.date(1518, 9, 9), '00:31', 'wakes', 'up'],
 [datetime.date(1518, 7, 21), '00:15', 'wakes', 'up'],
 [datetime.date(1518, 7, 30), '00:43', 'wakes', 'up'],
 [datetime.date(1518, 10, 8), '00:52', 'wakes', 'up'],
 [datetime.date(1518, 7, 26), '23:57', 'Guard', '#1051', 'begins', 'shift'],
 [datetime.date(1518, 7, 17), '00:00', 'Guard', '#2273', 'begins', 'shift'],
 [datetime.date(1518, 8, 18), '00:24', 'falls', 'asleep'],
 [datetime.date(1518, 9, 8), '00:02', 'falls', 'asleep'],
 [datetime.date(1518, 8, 19), '00:00', 'Guard', '#2917', 'begins', 'shift'],
 [da